In [75]:
import os
import pdb
import pandas as pd
import numpy as np
import ast_functions as ast

In [80]:
pd.set_option('display.max_columns', 20)
pd.set_option('display.max_rows', 10)

# Model: Decision Tree

### Read naive support vector classifier dataset

In [81]:
os.chdir('/Users/ethanchan/AST-ML/exported_data/')
# read dataset from naive support vector classifier
df = pd.read_csv('best_acc_&_alphas_no_0.csv')
# filter df using confidence level
filt = (df['confidence'] >= 10)
df = df[filt]
df

,accuracy,sensitivity,specificity,alpha,orientation,confidence
pk-2003,53.44,52.97,63.64,82962.5,1,10
pk-2004,55.49,54.96,58.70,120.0,0,13
pk-2005,59.10,56.16,61.14,98.0,0,14
pk-2006,64.85,64.71,75.00,174958.5,1,12
pk-2007,55.52,69.09,52.46,113.5,0,12
...,...,...,...,...,...,...
pk-8093,57.72,67.74,54.35,654.5,1,10
pk-8892,55.84,56.43,55.36,1923.5,1,12
pk-8893,58.33,55.31,72.00,2005.0,1,11
pk-9627,55.66,55.37,63.64,1136.5,0,13


### Read ms-ast dataset

In [82]:
ms_ast_df = pd.read_csv(f'pd_df_bin=1.csv')
ms_ast_df

,S-1,S-2,S-3,S-4,S-5,S-6,S-7,S-8,S-9,S-10,...,R-1259,R-1260,R-1261,R-1262,R-1263,R-1264,R-1265,R-1266,R-1267,R-1268
pk-2000,0,0,0,0,0,0,0,0,0,103171,...,0,0,0,0,0,0,0,0,0,0
pk-2001,0,0,0,0,60537,59179,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
pk-2002,0,0,0,0,0,0,0,118515,132154,0,...,0,0,0,0,0,201394,0,0,21674,0
pk-2003,0,0,0,0,0,0,39258,0,0,0,...,0,0,0,0,0,0,0,0,0,0
pk-2004,0,0,26495,0,0,0,0,0,0,0,...,28454,0,0,0,25316,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
pk-19996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
pk-19997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
pk-19998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
pk-19999,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Generate decision tree input feature dataset

In [83]:
dt_input_df = pd.DataFrame(columns=ms_ast_df.columns.values, index=df.index.values)
for idx in df.index.values:
    dt_input_df.loc[idx, :] = ms_ast_df.loc[idx, :]
dt_input_df.T

,pk-2003,pk-2004,pk-2005,pk-2006,pk-2007,pk-2008,pk-2009,pk-2010,pk-2013,pk-2014,...,pk-7566,pk-7567,pk-7568,pk-8091,pk-8092,pk-8093,pk-8892,pk-8893,pk-9627,pk-9628
S-1,0,0,0,0,0,0,0,1753098,0,0,...,0,0,0,0,0,152545,99597,0,0,0
S-2,0,0,0,0,0,0,0,0,0,0,...,0,0,16349912,0,0,8429219,0,9806929,0,31528448
S-3,0,26495,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,5141960,0,0
S-4,0,0,0,0,0,811574,0,0,0,0,...,0,2282831,0,0,799688,0,601221,0,0,4992858
S-5,0,0,0,0,0,127627,0,132314,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
R-1264,0,0,0,1080402,0,0,0,0,0,0,...,0,0,0,0,0,0,76932,0,0,0
R-1265,0,0,0,427705,0,0,0,0,0,0,...,0,1783498,0,0,0,0,0,44602,584850,0
R-1266,0,0,0,804144,0,0,0,0,674851,0,...,0,70012488,0,0,0,0,0,0,0,0
R-1267,0,0,0,60678,0,0,29928,0,172635,0,...,0,2538276,0,0,0,0,294556,0,0,2285660
